# Определение токсичных комментариев

## Описание проекта

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Предстоит обучить модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Модель должна иметь значение метрики качества *F1* не меньше 0.75.

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
# Импортируем все нужные модули.
import pandas as pd
import nltk
import re

# Игнорируем предупреждения.
import warnings
warnings.filterwarnings('ignore')

from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# Создаём константу для использования в параметрах вроде random_state.
SEED = 12345

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [2]:
# Загружаем датасет и выводим информацию о нём.
data = pd.read_csv('/datasets/toxic_comments.csv')
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


<div class="alert alert-block alert-success">
<b>Успех:</b> Загрузка данных и первичный осмотр проведены верно.
</div>

Создадим функции для очистки текста от лишних символов, и лемматизации.

In [3]:
def clean_text(row):
    '''
    Принимает на вход строку датасета, и очищает его,
    оставляя только пробелы и буквы, 
    возвращает очищенную строку.
    '''
    row = re.sub(r"(?:\n|\r)", " ", row)
    row = re.sub(r"[^a-zA-Z ]+", "", row).strip()
    row = row.lower()
    return row

In [4]:
# Применяем функции к датасету и проверяем, сработали ли они.
data['text'] = data['text'].apply(clean_text)
corpus = data['text'].values
corpus

array(['explanation why the edits made under my username hardcore metallica fan were reverted they werent vandalisms just closure on some gas after i voted at new york dolls fac and please dont remove the template from the talk page since im retired now',
       'daww he matches this background colour im seemingly stuck with thanks  talk  january   utc',
       'hey man im really not trying to edit war its just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info',
       ...,
       'spitzer   umm theres no actual article for prostitution ring   crunch captain',
       'and it looks like it was actually you who put on the speedy to have the first version deleted now that i look at it',
       'and  i really dont think you understand  i came here and my idea was bad right away  what kind of community goes you have bad ideas go away instead of helping rewrite them

# 2. Обучение

In [5]:
# Установим стоп-слова и вычислим TF-IDF.
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

# В качестве признака установим TF-IDF, а целевым признаком будем столбец 'toxic'.

features = corpus
target = data['toxic'].values

# Разделим признаки на обучающую, тестовую и валидационную выборки.
features_train, features_test, target_train, target_test = train_test_split(
    features, target, shuffle=False, test_size=.25, random_state=SEED)

features_train = count_tf_idf.fit_transform(features_train)
features_test = count_tf_idf.transform(features_test)

В качестве модели будем использовать только LogisticRegression, так как остальные не отрабатывают такое количество признаков за приемлимое время.

In [6]:
%%time
# Готовим пайплайн для модели.
pipe = Pipeline([
    (
    ('model', LogisticRegression(random_state=SEED, solver='liblinear', max_iter=100))
    )
])

param_grid = [
        {
            'model': [LogisticRegression(random_state=SEED, solver='liblinear')],
            'model__penalty' : ['l1', 'l2'],
            'model__C': list(range(1,15,3))
        }
]

# Прогоняем модель через GridSearchCV для нахождения лучших параметров.
grid = GridSearchCV(pipe, param_grid=param_grid, scoring='f1', cv=3, verbose=True)
best_grid = grid.fit(features_train, target_train)
print('Best parameters is:', grid.best_params_)
print('Best score is:', grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  3.7min finished


Best parameters is: {'model': LogisticRegression(C=4, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=12345, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False), 'model__C': 4, 'model__penalty': 'l1'}
Best score is: 0.7704474812157711
CPU times: user 2min 27s, sys: 1min 18s, total: 3min 46s
Wall time: 3min 46s


F1 больше 0.75, проверим на тестовой выборке.

# 3. Выводы

In [8]:
model = grid.best_estimator_
predicted_test = model.predict(features_test)
f1_score(target_test, predicted_test)

0.7826438485379328

F1-мера соответствует требуемуму значению. В теории, на других моделях классификации она могла быть больше, но они не работают с требуемой скоростью для такого количества данных.

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны

<a href='#description'>Назад к описанию</a>